#### Group members

Mostafa Allahmoradi - 9087818
Jarius Bedward - 8841640

## Imports


In [1]:

from gensim.models import Word2Vec

## Collection, Tokenizer, Normalization Pipeline

## Implement a Word2Vec predictive model using the knowledge corpus.

In [ ]:
#Words2Vec

model_w2v = Word2Vec(
    sentences=tokenized_corpus, # the tokenized corpus must be a list of lists
    vector_size=100,    #size of embedding
   window=5,        #context window
   min_count=1,     #keep all words (for demo purpose
    workers=4,         #choose how much cpu coreses use
  sg = 1            # number of skip-grams = 1 since this is small data 1
)

#train model
model_w2v.train(tokenized_corpus, total_examples=len(tokenized_corpus), epochs=20)

# ex: check similar words
# Example: check similar words
model_w2v.wv.most_similar("nlp", topn=3)



NameError: name 'tokenized_corpus' is not defined

#### Implement a GloVe count-based model using the knowledge corpus.

In [ ]:
!pip install glove-python-binary

from glove import Glove, Corpus
import numpy as np

sentences = [['this', 'is', 'an', 'example'], ['glove', 'is', 'awesome']]

corpus = Corpus()
corpus.fit(sentences, window=5)

glove_model = Glove(no_components=100, learning_rate=0.05)
glove_model.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove_model.add_dictionary(corpus.dictionary)

print(glove_model.word_vectors[glove_model.dictionary['glove']])
print(glove_model.most_similar('glove'))

#Download Glove Pretrained Embeddings From: http://nlp.stanford.edu/data/glove.6B.zip  

def embedding_for_vocab(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1
      
    # Adding again 1 because of reserved 0 index
    embedding_matrix_vocab = np.zeros((vocab_size, embedding_dim))
  
    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index.index(word)
                embedding_matrix_vocab[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]
                
    return embedding_matrix_vocab
  
# matrix for vocab: tokenized_words
embedding_dim = 50
embedding_matrix_vocab = embedding_for_vocab('../glove.6B.50d/glove.6B.50d.txt', tokenized_words, embedding_dim)
  
print("Dense vector for first word is => ", embedding_matrix_vocab[1])

## 🧠 Learning Objectives
- Teams of 2 (individual evaluation in class).
- Implement **Word2Vec**  and **GloVe** using real-world data during the NLP process.
- Build **Jupyter Notebooks** with well-structured code and clear Markdown documentation.
- Use **Git and GitHub** for collaborative version control and code sharing.
- Identify and articulate coding issues ("**talking points**") and insert them directly into markdown comments.


## 🧩 Workshop Structure (In Class)
1. **Set up teams of 2 people** – Read and understand the workshop, plus submission instructions. Seek assistance if needed.
2. **Jupyter Notebook Development** *(In class)* – NLP Pipeline (if needed) and Probabilistic Model method implementations + Markdown documentation (work as teams)
3. **Push to GitHub** – Teams commit and push the notebook. **Make sure to include your names so it is easy to identify the team that developed the code**.
4. **Instructor Review** - The instructor will go around in class, take notes, and provide coaching as needed, during the **Peer Review Round**


## 💻 Submission Checklist
- ✅ `EmbeddingClusteringVectorizationWorkshop.ipynb` with:
  - Demo code: Document Collection, Tokenizer, Normalization Pipeline on a relevant corpus.
  - Demo code: Implement a Word2Vec predictive model using the knowledge corpus.
  - Demo code: Implement a GloVe count-based model using the knowledge corpus.
  - Markdown explanations for each major step
  - In a table that compare **Word2Vec** against **GloVe** in the context of the use case that makes use of the knowledge corpus.
- ✅ `README.md` with:
  - Dataset description
  - Team member names
  - Link to the dataset and license (if public)
- ✅ GitHub Repo:
  - Public repo named `EmbeddingClusteringVectorizationWorkshop`
  - **Markdowns and meaningful talking points**